In [1]:
import os
import numpy as np
from os.path import dirname
from pandas import DataFrame, read_csv, concat
ROOT_DIR = dirname(dirname(os.path.realpath('__file__')))

## Section 1: Task appraisals

In [2]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load and prepare self-report data (experiment 1).
m1 = concat([read_csv(os.path.join(ROOT_DIR, 'study01', 'data', s, 'metadata.csv')) for s in ['s1','s2','s3','s4']])
reject = read_csv(os.path.join(ROOT_DIR, 'study01', 'data', 's1', 'reject.csv'))
m1 = m1[m1.subject.isin(reject.query('reject==0').subject)].reset_index(drop=True)

## Load and prepare self-report data (experiment 2).
m2 = concat([read_csv(os.path.join(ROOT_DIR, 'study02', 'data', s, 'metadata.csv')) for s in ['s1','s2','s3']])
reject = read_csv(os.path.join(ROOT_DIR, 'study02', 'data', 's1', 'reject.csv'))
m2 = m2[m2.subject.isin(reject.query('reject==0').subject)].reset_index(drop=True)

## Concatenate DataFrames.
cols = ['subject','session','difficulty','fun','clarity']
data = concat([m1[cols].assign(study=1), m2[cols].assign(study=2)])

## Reverse-code difficulty.
data['difficulty'] = 6 - data['difficulty']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Report Table S02.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Melt DataFrame.
data = data.melt(id_vars=['study','subject','session'], var_name='item', value_name='response')

## Summarize data.
f = lambda x: '%0.1f (%0.1f)' %(x.mean(), x.std())
table = data.pivot_table('response', ['item','study'], 'session', aggfunc=f).fillna('-')

## Format labels.
table = table.rename(index={'clarity': 'Clarity', 'difficulty': 'Difficulty', 'fun': 'Fun'},
                     columns={1: 'Day 0', 2: 'Day 3', 3: 'Day 14', 4: 'Day 28'})
table = table.loc[['Difficulty','Fun','Clarity']]

## Report table.
table

session               Day 0      Day 3     Day 14     Day 28
item       study                                            
Difficulty 1      2.5 (1.1)  2.2 (0.9)  2.1 (1.0)  1.8 (0.8)
           2      3.3 (1.1)  3.0 (1.1)  3.1 (1.1)          -
Fun        1      3.9 (0.9)  4.0 (1.0)  3.9 (0.9)  4.1 (0.9)
           2      3.9 (0.9)  4.1 (0.8)  4.0 (0.8)          -
Clarity    1      4.8 (0.6)  4.9 (0.4)  5.0 (0.2)  4.9 (0.3)
           2      4.9 (0.5)  4.9 (0.2)  4.9 (0.2)          -